# Table of Contents
 <p><div class="lev1 toc-item"><a href="#MIDS---w261-Machine-Learning-At-Scale" data-toc-modified-id="MIDS---w261-Machine-Learning-At-Scale-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MIDS - w261 Machine Learning At Scale</a></div><div class="lev2 toc-item"><a href="#Assignment---HW3---Supplement" data-toc-modified-id="Assignment---HW3---Supplement-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Assignment - HW3 - Supplement</a></div><div class="lev2 toc-item"><a href="#INSTRUCTIONS-for-SUBMISSION" data-toc-modified-id="INSTRUCTIONS-for-SUBMISSION-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>INSTRUCTIONS for SUBMISSION</a></div><div class="lev2 toc-item"><a href="#CONFIGURATION" data-toc-modified-id="CONFIGURATION-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>CONFIGURATION</a></div><div class="lev2 toc-item"><a href="#DATASETS" data-toc-modified-id="DATASETS-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>DATASETS</a></div><div class="lev2 toc-item"><a href="#HW3.4.-Computing-Confidence---Pairs" data-toc-modified-id="HW3.4.-Computing-Confidence---Pairs-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>HW3.4. Computing Confidence - Pairs</a></div><div class="lev3 toc-item"><a href="#Count-&amp;-Sort-Pairs" data-toc-modified-id="Count-&amp;-Sort-Pairs-151"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Count &amp; Sort Pairs</a></div><div class="lev3 toc-item"><a href="#Sanity-check-for-support." data-toc-modified-id="Sanity-check-for-support.-152"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Sanity check for support.</a></div>

# MIDS - w261 Machine Learning At Scale
__Course Lead:__ Dr James G. Shanahan (__email__ Jimi via  James.Shanahan _AT_ gmail.com)

## Assignment - HW3 - Supplement


---
__Name:__  *Your Name Goes Here*   
__Class:__ MIDS w261 (Section *Your Section Goes Here*, e.g., Fall 2016 Group 1)     
__Email:__  *Your UC Berkeley Email Goes Here*@iSchool.Berkeley.edu     
__StudentId__  123457    __End of StudentId__     

__NOTE:__ please replace `1234567` with your student id above      

## INSTRUCTIONS for SUBMISSION

This homework can be completed locally on your computer. __Please submit your notebook to your classroom github repository 24 hours prior to the next live session.__ 

## CONFIGURATION
Before starting your homework, run the following cells to confirm your setup.

In [7]:
# general imports
import os
import re
import sys
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# automatically reload modules 
%reload_ext autoreload
%autoreload 2

In [4]:
# print some configuration details for future replicability.
print 'Python Version: %s' % (sys.version.split('|')[0])
hdfs_conf = !hdfs getconf -confKey fs.defaultFS ### UNCOMMENT ON DOCKER
#hdfs_conf = !hdfs getconf -confKey fs.default.name ### UNCOMMENT ON ALTISCALE
print 'HDFS filesystem running at: \n\t %s' % (hdfs_conf[0])

Python Version: 2.7.13 
HDFS filesystem running at: 
	 hdfs://quickstart.cloudera:8020


In [11]:
# create an HDFS directory for this assignment
!hdfs dfs -mkdir hw3

__[OPTIONAL]:__ Save yourself some typing by defining global variables for paths we'll reuse frequently (like the hadoop-streaming jar file and the HDFS directory for your output).

In [22]:
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/hw3"
HOME_DIR = os.getcwd() 

__[OPTIONAL]:__ Fix chrome formatting. _The cell below implements a quick hack based on [this stackoverflow thread](http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line) to fix [this known issue](https://github.com/mathjax/MathJax/issues/1300) with Mathjax formatting in Chrome (a rounding issue adds a border to the right of mathjax markup)._

In [ ]:
%%javascript
$('.math>span').css("border-left-color","transparent")

## DATASETS
For this homework we be using a few different datasets. These include:
* __The Consumer Complaints Dataset [available here](https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0) -__ _consists of diverse consumer complaints which have been reporte across the United States regarding various types of loans. We'll use this in HW 3.1 to learn about Hadoop Counters and then in HW 3.2 where we count words and compute relative frequencies._

* __The Product Purchase Dataset [available here](https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0) -__ _consists of data about products sold online and customer browsing behavior. We'll use this data in 3.3 to perform a shopping cart analysis._

Follow the directions below to load/create each of these datasets. You may want to familiarize yourself with their contents before proceeding to the homework questions.

__`Consumer_Complaints.csv`__  
The dataset consists of records of the form:  
>Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Download the csv file to your current directory using the dropbox link provided above. You can do this manually or with `curl` or `wget` as we did in HW2. Then use the following cell to put the data into HDFS.

In [15]:
# Download the data and remove the header
!curl -L -O  https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv
!head -n 1 Consumer_Complaints.csv > CC_header.csv
!tail -n +2 Consumer_Complaints.csv > CC_records.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48.5M  100 48.5M    0     0  3644k      0  0:00:13  0:00:13 --:--:-- 4355k --:--:--  0:00:01 --:--:--     0


In [24]:
# take a look at the first two lines
!head -n 2 Consumer_Complaints.csv

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,


In [ ]:
# Put the data into HDFS - adjust path or filename as needed
!hdfs dfs -put CC_records.csv {HDFS_DIR}/Consumer_Complaints.csv

__`ProductPurchaseData.txt`__  
Notes about the data format:
> Each line in this dataset represents a browsing session of a customer. On each line, each string of 8 characters represents the id of an item browsed during that session. For example:  
`FRO11987 ELE17451 ELE89019 SNA90258 GRO99222`  
> would represent a single customer's browsing session in which they viewed 5 products.  

Download the csv file to your current directory using the dropbox link provided above. You can do this manually or with `curl` or `wget` as we did in HW2. Take a look at the first few lines of the file, then use the following cell to put the data into HDFS.

In [25]:
# Downloading the data
!curl -L -O  https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3377k  100 3377k    0     0  1621k      0  0:00:02  0:00:02 --:--:-- 4307k


In [26]:
# Take a peak at the data
!head -n 5 ProductPurchaseData.txt

FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


In [28]:
# Put the data into HDFS - adjust path or filename if needed
!hdfs dfs -put ProductPurchaseData.txt {HDFS_DIR}/ProductPurchaseData.txt

## HW3.4. Computing Confidence - Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

The relative frequency of a bigram (AKA itemset of size 2) $(w_i, w_j)$ relative to its first word $w_i$ is the number of occurrences of the pair $(w_i, w_j)$ related to the number of occurrences of the word $w_i$. For instance, if the word "euro" followed by the word "crisis" occurs 10 times and the word "euro" occurs 20 times in total, we say that the relative frequency of the pair ("euro","crisis") is $\frac{10}{20}$ = 0.5.

$$ f(B|A) = \frac{count(A,B)}{count(A)} = \frac{count(A,B)}{\sum_{B'} count(A,B')}$$

__In association rule mining this is refered to as confidence. __   
__Ref: https://www-users.cs.umn.edu/~kumar/dmbook/ch6.pdf pg.3-4__


List the top 50 product pairs with corresponding support count (aka frequency), and confidence in decreasing order of confidence for frequent (100>count) itemsets of size 2. 


Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Feel free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner, and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, confidence



Fix the ordering of the pairs lexicographically (left to right), and break ties in confidence (between pairs, if any exist) by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.





In [11]:
%%writefile pairsMapper.py
#!/usr/bin/env python
from __future__ import division
import sys
import itertools

############################################################
# Using order inversion, we are able to count the total
# number of the first word in each pair to send to the reducer 
# the relative frequencies.
############################################################


sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

total = 0

for line in sys.stdin:
    line = line.strip()
    words = line.split()
    total += 1
    for subset in itertools.combinations(sorted(set(words)), 2):
        print '%s\t%s\t%s' % (subset[0], subset[1], 1)
    for word in words:
        print '%s\t%s\t%s' % (word,"!", 1)          


Writing pairsMapper.py


In [34]:
%%writefile pairsReducer.py
#!/usr/bin/env python
from __future__ import division
import sys

sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")


cur_key = None
cur_key_count = 0

cur_pair = None
cur_pair_count = 0

support_count = 100

for line in sys.stdin:
    line = line.strip()
    A,B,count = line.split("\t")
    
    if A == cur_key:
      if B == "!":
        cur_key_count += int(count)
        
      elif A+"-"+B == cur_pair:
        cur_pair_count += int(count)
    
      else:
        if cur_pair:
          if cur_pair_count >= support_count:
              print '%s\t%s\t%s' % (cur_pair,cur_pair_count,str(cur_pair_count/cur_key_count))
        
        cur_pair = A+"-"+B
        cur_pair_count = int(count)
        
    else:
      cur_key = A
      cur_key_count = int(count)

        
if cur_pair_count >= support_count:
      print '%s\t%s\t%s' % (cur_pair,cur_pair_count,str(cur_pair_count/cur_key_count))

Overwriting pairsReducer.py


### Count & Sort Pairs

In [13]:
!hdfs dfs -ls /user/koza

Found 3 items
drwxr-xr-x   - koza supergroup          0 2017-09-16 17:07 /user/koza/HW2
drwxr-xr-x   - koza supergroup          0 2017-07-19 09:42 /user/koza/graph
drwxr-xr-x   - koza supergroup          0 2017-07-19 09:41 /user/koza/tmp


In [14]:
!hdfs dfs -mkdir /user/koza/hw3
!hdfs dfs -mkdir /user/koza/hw3/3.4
!hdfs dfs -mkdir /user/koza/hw3/3.4/pairs

In [18]:
!hdfs dfs -put  ProductPurchaseData.txt /user/koza/hw3/ProductPurchaseData.txt

In [20]:
!hdfs dfs -ls /user/koza/hw3

Found 2 items
drwxr-xr-x   - koza supergroup          0 2017-09-23 16:31 /user/koza/hw3/3.4
-rw-r--r--   1 koza supergroup    3458517 2017-09-23 16:41 /user/koza/hw3/ProductPurchaseData.txt


In [53]:
import time


# Get counts:
start = time.time()
!hdfs dfs -rm -r /user/koza/hw3/3.4/pairs
!hadoop jar /usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
    -D stream.num.map.output.key.field=3 \
    -D stream.map.output.field.separator="\t" \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -files pairsMapper.py,pairsReducer.py \
    -mapper pairsMapper.py \
    -reducer pairsReducer.py \
    -input /user/koza/hw3/ProductPurchaseData.txt -output /user/koza/hw3/3.4/pairs
end = time.time()
countTime = end-start



# After calculating all the counts, We can do a secondary sort:
start = time.time()
!hdfs dfs -rm -r /user/koza/hw3/3.4/pairs_sorted
!hadoop jar /usr/local/Cellar/hadoop/2.7.2/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
    -D stream.num.map.output.key.field=3 \
    -D stream.map.output.field.separator="\t" \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
    -mapper /bin/cat \
    -reducer /bin/cat \
    -input /user/koza/hw3/3.4/pairs/* -output /user/koza/hw3/3.4/pairs_sorted

end = time.time()
sortTime = end-start

17/09/23 17:36:17 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/koza/hw3/3.4/pairs
packageJobJar: [/var/folders/2f/rb8qqgd55bl77zgchyxsfl7h0000gp/T/hadoop-unjar808882282825266836/] [] /var/folders/2f/rb8qqgd55bl77zgchyxsfl7h0000gp/T/streamjob5542986570983811443.jar tmpDir=null
17/09/23 17:36:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/09/23 17:36:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/09/23 17:36:20 INFO mapred.FileInputFormat: Total input paths to process : 1
17/09/23 17:36:20 INFO mapreduce.JobSubmitter: number of splits:2
17/09/23 17:36:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1506180544228_0021
17/09/23 17:36:20 INFO impl.YarnClientImpl: Submitted application application_1506180544228_0021
17/09/23 17:36:20 INFO mapreduce.Job: The url to track the job: http://localhost:8088/proxy/application_1506180544228_0021/

In [55]:
print "-------  Time to count and sort pairs  --------\n"
print "Count time:\t %f sec\nSort time:\t %f sec\nTotal time:\t %f sec " % (countTime,sortTime,countTime+sortTime)

-------  Time to count and sort pairs  --------

Count time:	 44.506754 sec
Sort time:	 22.396491 sec
Total time:	 66.903245 sec 


In [2]:
print "\n------------- Most frequent pairs --------------\n"
print "\nPair                Support count     Confidence \n"
!hdfs dfs -cat /user/koza/hw3/3.4/pairs_sorted/part-00000 | head -n 50
print "\n----- Least frequent but not less than 100 ------\n"
!hdfs dfs -cat /user/koza/hw3/3.4/pairs_sorted/part-00000 | tail -n 10


------------- Most frequent pairs --------------


Pair                Support count     Confidence 

DAI62779-ELE17451	1592	0.238788060597	
FRO40251-SNA80324	1412	0.363823756764	
DAI75645-FRO40251	1254	0.458333333333	
FRO40251-GRO85051	1213	0.312548312291	
DAI62779-GRO73461	1139	0.170841457927	
DAI75645-SNA80324	1130	0.413011695906	
DAI62779-FRO40251	1070	0.160491975401	
DAI62779-SNA80324	923	0.138443077846	
DAI62779-DAI85309	918	0.137693115344	
ELE32164-GRO59710	911	0.31953700456	
DAI62779-DAI75645	882	0.132293385331	
FRO40251-GRO73461	882	0.227261015202	
DAI62779-ELE92920	877	0.131543422829	
FRO40251-FRO92469	835	0.215150734347	
DAI62779-ELE32164	832	0.124793760312	
DAI75645-GRO73461	712	0.260233918129	
DAI43223-ELE32164	711	0.551162790698	
DAI62779-GRO30386	709	0.106344682766	
ELE17451-FRO40251	697	0.179870967742	
DAI85309-ELE99737	659	0.287396423899	
DAI62779-ELE26917	650	0.0974951252437	
GRO21487-GRO73461	631	0.298345153664	
DAI62779-SNA45677	604	0.0905954702265	
ELE17451-SNA803

### Sanity check for support.
If the support (1592) divided by the count of 'DAI62779' == 0.238788060597, then algorithm is OK

In [63]:
!grep 'DAI62779' ProductPurchaseData.txt | wc -l

    6667


In [64]:
from __future__ import division
1592/6667

0.23878806059697016

*Report the compute time for the Pairs job:*     

```
Count time:	 44.506754 sec
Sort time:	 22.396491 sec
Total time:	 66.903245 sec  
```

*Describe the computational setup used:*    
   
MacBook Pro, 2.5 GHz Intel Core i7  (4 Cores) 
16 GB 1600 MHz DDR3   

Launched map tasks=2   
Launched reduce tasks=1   

!["counters 3.4"](http://candpgeneration.com/images/counters3.4.png)
